In [1]:
import numpy as np
import pymannkendall as mk

In [4]:
pre=np.array([12,24,43,53,130,102,140,159,133,84,42,26,29,46,37,66,71,103,150,143,129,75,33,12])

例一：计算sens斜率并进行原始Mann-Kendall检验

In [5]:
result = mk.original_test(pre,alpha=0.05)

#pre是降水数据，alpha是置信区间
#OUT变量对应上述
'''
Mann_Kendall_Test(
     trend='no trend',
     h=False,
     p=0.5851588651847224, 
     z=0.5458648159491473,
     Tau=0.08333333333333333,
     s=23.0, var_s=1624.3333333333333,
     slope=0.8500000000000001,
     intercept=58.725)'''

"\nMann_Kendall_Test(\n     trend='no trend',\n\xa0\xa0\xa0\xa0\xa0h=False,\n\xa0\xa0\xa0\xa0\xa0p=0.5851588651847224,\xa0\n     z=0.5458648159491473,\n     Tau=0.08333333333333333,\n     s=23.0, var_s=1624.3333333333333,\n     slope=0.8500000000000001,\n\xa0\xa0\xa0\xa0\xa0intercept=58.725)"

当只进行Sen's斜率计算时：

In [9]:
result = mk.sens_slope(pre)
print(result)

Sens_Slope_Test(slope=0.8500000000000001, intercept=58.725)


例二：计算季节Sen's斜率并进行季节MK检验

In [10]:
result = mk.seasonal_test(pre,alpha=0.05,period=12)
print(result)

Seasonal_Mann_Kendall_Test(trend='no trend', h=False, p=0.7728299926844475, z=-0.2886751345948129, Tau=-0.16666666666666666, s=-2.0, var_s=12.0, slope=-5.0, intercept=73.29166666666667)


当只进行季节Sen's斜率计算时：

In [12]:
result = mk.seasonal_sens_slope(pre,period=12)
print(result)

Seasonal_Sens_Slope_Test(slope=-5.0, intercept=73.29166666666667)
